In [ ]:
!pip install simpletransformers -q

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import classification_report

In [95]:
df = pd.read_csv("train.csv",usecols=["text","target"]);df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [96]:
# Simpletransformers expects it this way
df["labels"] = df["target"]
del df["target"]
df.head()

,text,labels
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test_size = len(df) * 3 // 10
test_idx = np.random.choice(df.labels.index,test_size,replace=False)

In [ ]:
df_eval = df.iloc[test_idx]
df_train = df.iloc[~test_idx]

In [104]:
args = {'reprocess_input_data': True, 'overwrite_output_dir': True,
            'num_train_epochs': 1,
            'max_seq_length': 300, 'evaluate_during_training': True}

model = ClassificationModel("roberta", "roberta-base",args=args)

In [107]:
# Run This for installing Apex
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [108]:
!sh setup.sh -q

Cloning into 'apex'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 6672 (delta 45), reused 32 (delta 17), pack-reused 6596
Receiving objects: 100% (6672/6672), 13.75 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (4433/4433), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-akhpk7t2
Created temporary directory: /tmp/pip-req-tracker-9e_gp21b
Created requirements tracker '/tmp/pip-req-tracker-9e_gp21b'
Created temporary directory: /tmp/pip-install-ln_npvpa
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-8hnk1jgz
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-9e_gp21b'
    Running setup.py (path:/tmp/pip-req-build-8hnk1jgz/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-8hnk1jgz/pip-

In [109]:
model.train_model(df_train,eval_df=df_eval)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.631216

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.681008Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.519577Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.395806Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.934174

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.318692Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 0.264761


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [111]:
predictions, raw_outputs = model.predict(df_eval.text.values)

In [116]:
print(classification_report(df_eval.labels,predictions))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1320
           1       0.85      0.76      0.80       963

    accuracy                           0.84      2283
   macro avg       0.84      0.83      0.83      2283
weighted avg       0.84      0.84      0.84      2283

